In [37]:
import os,django
import gensim
import pandas as pd
from gensim.models.doc2vec import Doc2Vec

In [2]:
print (os.getcwd())

/Users/litianqi/school/TLCAS/papers/static/doc2vec_recommendation


### 获取上层目录，用于读取数据集

In [5]:
print (os.path.abspath(os.path.join(os.getcwd(), "../../..")))

/Users/litianqi/school/TLCAS


In [3]:
os.chdir(os.path.abspath(os.path.join(os.getcwd(), "../../..")))

In [4]:
print (os.getcwd())

/Users/litianqi/school/TLCAS


### 读取数据库

In [11]:
os.environ.setdefault("DJANGO_SETTINGS_MODULE", "TLCAS.settings")
os.environ["DJANGO_ALLOW_ASYNC_UNSAFE"] = "true"
django.setup()

In [12]:
from papers.models import *

In [14]:
Papers = list(PaperInfo.objects.all())

In [18]:
Papers[234].pk

235

In [19]:
Papers[0].paper_title

'Where and Why Users "Check In"'

In [20]:
Papers[0].abstract

'The emergence of location based social network (LBSN) services makes it possible to study individuals?mobility patterns at a fine-grained level and to see how they are impacted by social factors. In this study we analyze the check-in patterns in LBSN and observe significant temporal clustering of check-in activities. We explore how self-reinforcing behaviors, social factors, and exogenous effects contribute to this clustering and introduce a framework to distinguish these effects at the level of individual check-ins for both users and venues. Using check-in data from three major cities, we show not only that our model can improve prediction of future check-ins, but also that disentangling of different factors allows us to infer meaningful properties of different venues.'

In [22]:
Papers[0].paper_title + ' ' + Papers[0].abstract

'Where and Why Users "Check In" The emergence of location based social network (LBSN) services makes it possible to study individuals?mobility patterns at a fine-grained level and to see how they are impacted by social factors. In this study we analyze the check-in patterns in LBSN and observe significant temporal clustering of check-in activities. We explore how self-reinforcing behaviors, social factors, and exogenous effects contribute to this clustering and introduce a framework to distinguish these effects at the level of individual check-ins for both users and venues. Using check-in data from three major cities, we show not only that our model can improve prediction of future check-ins, but also that disentangling of different factors allows us to infer meaningful properties of different venues.'

### 构建训练集

In [29]:
TaggedDocument = gensim.models.doc2vec.TaggedDocument
X_train = []
for paper in Papers:
    document = TaggedDocument(paper.paper_title + ' ' + paper.abstract, tags=[paper.pk])
    X_train.append(document)

In [39]:
# Doc2Vec = gensim.models.doc2vec
model = Doc2Vec(X_train, min_count=1, window=3, size=300, sample=1e-3, nagative=5, workers=4)
model_result = model.train(X_train, total_examples=model.corpus_count, epochs=10)

In [42]:
model_result 


In [46]:
from gensim.test.utils import get_tmpfile
fname = get_tmpfile("doc2vec_model_1_202010201337")
model.save(fname)

In [62]:
inferred_vector_dm = model.infer_vector(Papers[123].abstract.split(','))
sims = model.docvecs.most_similar([inferred_vector_dm], topn=10)

In [63]:
sims

[(24335, 0.14039349555969238),
 (21979, 0.13383471965789795),
 (4584, 0.13353264331817627),
 (3447, 0.12102784216403961),
 (1970, 0.1195373684167862),
 (28103, 0.11825913190841675),
 (72, 0.11721032857894897),
 (6, 0.11718027293682098),
 (25293, 0.11663002520799637),
 (26304, 0.11581048369407654)]